## Text Feature Extractor
this notebook generates for each text some additional/handcrafted features characterizing its named entity composition and their relative relative position (entity distance):distance is the number of words separating 2 entities (it can be negative)
* number of sentences
* number of words
* distinct count of drug name entities
* distinct count of active ingredient entities
* count of question marks (typically to identify specifically multi-intent label)
* individual count of interrogative pronoun entities (one column per pronoun: quand, qui, quoi, ou, comment, pourquoi, combien, quel(s|le,..)
* distinct count of time entities (eg: jours, après midi, soir, année, 12h, mardi, samedi, temps....)
* distinct count of quantity entities (eg: 5mg, 10ml, ...)
* count of association entities (eg: et, avec, ou, ...)
* distance between interrogative pronoun and drug name entities
* distance between active ingredient and drug name entities
* distance between quantity and drug name entities
* distance between time and drug name entities
* distance between question marks and drug name entities

these extracted features are stored into this [file](../../data/staging_data/text_extracted_features.csv) with ID column to join it with the former train dataset if required.

In [1]:
import pandas as pd
import numpy as np

XTrain = pd.read_csv('../../data/staging_data/mispelling_fixed_clean_input_train.csv', sep=',')

In [63]:
from collections import Counter
import re
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

def words(text):
    return re.findall(r'\w+', text.lower())

# value domain based named entities
drugNames = set(words(open('../../data/staging_data/drug_names.txt').read()))
ingredientNames = set(words(open('../../data/staging_data/ingredients.txt').read()))

# reg expression based named entities
questionMarkRegEx = r"[\?]+"
compiledquestionMarkRegEx = re.compile(questionMarkRegEx)

sentenceSeparatorRegEx = r"[\?]+|!|:|;|[\.]+"
#quiRegEx = r"[\s\?\.\!\:\;]?[Q|q]ui\s" qui term is ambiguous
combienRegEx = r"[\s\?\.\!\:\;]?[C|c]ombien\s"
pourquoiRegEx = r"[\s\?\.\!\:\;]?[P|p]ourquoi\s"
quandRegEx = r"[\s\?\.\!\:\;]?[Q|q]uand\s"
quoiRegEx = r"[\s\?\.\!\:\;]?[Q|q]uoi\s|[\s\?\.\!\:\;]?[Q|q]uel[l|s|le|les]?\s"
commentRegEx = r"[\s\?\.\!\:\;]?[C|c]omment\s"
interrogativePronounRegEx = combienRegEx + '|' + pourquoiRegEx + '|' + quandRegEx + '|' + quoiRegEx + '|' + commentRegEx
compiledPronounRegEx = re.compile(interrogativePronounRegEx)

# associative entity
ouRegEx = r"\sou\s"
etRegEx = r"\set\s"
avecRegEx = r"\savec\s"


# gold standard
timeRegEx = r"([D|d]emain|[H|h]ier|[M|m]atin|[M|m]idi|[S|s]oir|mois|jours|[D|d]epuis|semaine[s|S]|heure[s|S])( )*( |,|\.)"
quantityRegEx = r"[0-9]+\s?[m|k]?g|\s[m|k]?g[^a-zA-Zéàèî]|[0-9]+\s?[m|c]?l|\s[m|c]?l[^a-zA-Zéàèî]"

def getWordCount(text):
    words = word_tokenize(text, language='french')
    return len(words)

def getEntityCount(text, entityValueSet, lowerCase):
    ''' return distinct count of entity occurrences found from the text'''
    words = word_tokenize(text, language='french')
    if lowerCase:
        words = map(lambda x: x.lower(), words)
    return len(list(set(words) & entityValueSet))

def countWordsBefore(text, lastPosition):
    ''' count number of words before this position'''
    text = text[0:lastPosition]
    words = word_tokenize(text, language='french')
    return len(words)

def getWordIndexByReg(text, regEx):
    ''' return the word index of the first group matching the regexp'''
    iterables = list(regEx.finditer(text))
    positions = list(map(lambda x : x.span()[0], iterables))
    if len(positions) > 0:
        position = int(positions[0])
        return countWordsBefore(text, position)
    else:
        return None

def getWordIndexByValueSet(text, valueSet):
    ''' return the word index of the first token part of the value domain'''
    words = word_tokenize(text, language='french')
    wordIndex = 0
    for word in words:
        if word in valueSet:
            return wordIndex
        wordIndex = wordIndex + 1
    return None

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Jacques\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
XTrain['drugCount'] = XTrain['question'].map(lambda text: getEntityCount(text, drugNames, True))
XTrain['ingredientCount'] = XTrain['question'].map(lambda text: getEntityCount(text, ingredientNames, True))

In [4]:
XTrain['timeCount'] = XTrain['question'].map(lambda text : len(re.findall(timeRegEx, text)))
XTrain['quantitiesCount'] = XTrain['question'].map(lambda text : len(re.findall(quantityRegEx, text)))
XTrain['questionMarkCount'] = XTrain['question'].map(lambda text : len(re.findall(questionMarkRegEx, text)))
XTrain['sentenceCount'] = XTrain['question'].map(lambda text : 1 + len(re.findall(sentenceSeparatorRegEx, text)))
XTrain['wordCount'] = XTrain['question'].map(lambda text : getWordCount(text))

In [5]:
XTrain['combienCount'] = XTrain['question'].map(lambda text : len(re.findall(combienRegEx, text)))
XTrain['pourquoiCount'] = XTrain['question'].map(lambda text : len(re.findall(pourquoiRegEx, text)))
XTrain['quandCount'] = XTrain['question'].map(lambda text : len(re.findall(quandRegEx, text)))
XTrain['quoiCount'] = XTrain['question'].map(lambda text : len(re.findall(quoiRegEx, text)))
XTrain['commentCount'] = XTrain['question'].map(lambda text : len(re.findall(commentRegEx, text)))

In [6]:
XTrain['avecCount'] = XTrain['question'].map(lambda text : len(re.findall(avecRegEx, text)))
XTrain['etCount'] = XTrain['question'].map(lambda text : len(re.findall(etRegEx, text)))
XTrain['ouCount'] = XTrain['question'].map(lambda text : len(re.findall(ouRegEx, text)))

In [55]:
XTrain['interrogativeWordIndex'] = XTrain['question'].map(lambda text : getWordIndexByReg(text, compiledPronounRegEx))
XTrain['questionMarkIndex'] = XTrain['question'].map(lambda text : getWordIndexByReg(text, compiledquestionMarkRegEx))

In [58]:
XTrain['drug_WordIndex'] = XTrain['question'].map(lambda text : getWordIndexByValueSet(text, drugNames))
XTrain['drug_InterrogativeDistance'] = XTrain['interrogativeWordIndex'] - XTrain['drugWordIndex']
XTrain['drug_QuestionMarkDistance'] = XTrain['questionMarkIndex'] - XTrain['drugWordIndex']

In [62]:
pd.set_option("display.max_colwidth",20000)
XTrain[XTrain.questionMarkIndex > 0]

,ID,question,drugCount,ingredientCount,timeCount,quantitiesCount,questionMarkCount,sentenceCount,wordCount,combienCount,...,interrogativeWordIndex,drugWordIndex,drugInterrogativeDistance,questionMarkIndex,drug-WordIndex,drug-InterrogativeDistance,drug-QuestionMarkDistance,drug_WordIndex,drug_InterrogativeDistance,drug_QuestionMarkDistance
29,29,"bonjour, depuis presque sous effexor.on prescrit xanax cas crise angoisse car fait 3.mais voila depuis jours angoissée matin soir cela sans raison apparente.je peux presque plus rien avaler tellement nouée alors habitude mange comme vache.la moindre chose faire semble insurmontable provoque malaises..je sais plus quoi faire.je prends xanax jour alors medecin prescrit crises.malgré xanax mega angoissée non stoptout entourage remarqué collègues croient enceinte.ai peur devenir completement agoraphobe.de plus medecin vacances sais meme passer xanax dangereux.savez combien temps peut durer type crise ?quels risques vois psychologue depuis premiere crise donc presque an. merci lumieres..",1,0,3,0,1,14,96,1,...,45.0,7.0,38.0,82.0,7.0,38.0,75.0,7.0,38.0,75.0
138,138,rotarix effets secondaires ?..,1,0,0,0,1,3,5,0,...,NaN,0.0,NaN,3.0,0.0,NaN,3.0,0.0,NaN,3.0
648,648,"bonjour, fait moins deux ans ancien homéopathe prescrit gelsemium sempervirens troub digestif savait lié anxiété autre bref.. cet homéopathie rien servit régler problèmes digestion, continuais malgré être très dérangé bref. coup petit tube comme neuf, rest plein granule date limite 2010, peux donc utiliser pbs angoisses, sauf plus ordonnance sais plus comment prendre, pouvez aider enfait aimerais savoir quand faut prendre jour angoisse fois journée terminé arrête prendre veut dire traitement duré jour faut prenne veille angoisse ?car souvent angoisse avance quelque chose, faut prenne semaine entière avant angoisse svp aidez sais tout comment faire vendredi samedi justement deux journée très angoissantes..",1,1,0,0,1,5,111,0,...,57.0,9.0,48.0,84.0,9.0,48.0,75.0,9.0,48.0,75.0
1107,1107,bonjour! voila prend minidril depuis mois javais enchainé mois tout trés bien passé.maintenent reste semaine pilule doi faire passe jour part week end alors esque peu arreté pilule semaine avan fait joré pri semaine faire pause comme joré regle week end avant ?merci,1,0,2,0,1,4,45,0,...,NaN,4.0,NaN,43.0,4.0,NaN,39.0,4.0,NaN,39.0
1249,1249,sous prozac depuis plus mois toujours aussi mal mieux prozac zoloft seroplex ?deroxat,4,0,2,0,1,2,14,0,...,NaN,1.0,NaN,12.0,1.0,NaN,11.0,1.0,NaN,11.0
1447,1447,bonjour toutes !!voila juste petite question..je prend pilule mercilon depuis semaine règ durent habituellement jours tjrs finis meme deviennent très peu abondantes..est normal pilule augmenté durée règ ?merci avance,1,0,1,0,1,6,32,0,...,NaN,10.0,NaN,29.0,10.0,NaN,19.0,10.0,NaN,19.0
1694,1694,prend pillule belara depuis environ court cette période pris kilos. quelque deja ete dan smon cas cette pillule kilos ils facil perdent suite ?..,1,0,0,0,1,4,26,0,...,NaN,2.0,NaN,24.0,2.0,NaN,22.0,2.0,NaN,22.0
1823,1823,allergie triflucan gonflement lèvres bucca nez ?!!,1,0,0,0,1,4,9,0,...,NaN,1.0,NaN,6.0,1.0,NaN,5.0,1.0,NaN,5.0
2067,2067,"prends varnoline continue. varnoline première contraception juste jamais cyc réguliers, donc quelques peurs approche fin première plaquette. dois commencer deuxieme plaquette comme prévu type varnoline, encore règ fin première plaquette ?oui contraire dois attendre arrivée règ avant commencer suivante pense couperait effet nan",1,0,0,0,1,4,48,0,...,NaN,1.0,NaN,34.0,1.0,NaN,33.0,1.0,NaN,33.0
2242,2242,bonjour toutes voila mois jai pris duphaston car suite arret pilule navais plus regles.le octobre jai reg commes prevu depuis hier jai pertes sang croire jai reg vont arriver.jaimerer savoir normal ?merci reponses,1,0,2,0,1,4,34,0,...,NaN,6.0,NaN,31.0,6.0,NaN,25.0,6.0,NaN,25.0


In [53]:
# save the extraction result
SavedTrain = XTrain.drop(["question"], axis=1)
SavedTrain.to_csv("../../data/staging_data/text_extracted_features.csv", index=None)